<a href="https://colab.research.google.com/github/OBILISETTVYSHNAVI/travel-recommendation/blob/main/Travel_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd
import numpy as np
import gradio as gr
import pickle
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import accuracy_score

# Load datasets
df1 = pd.read_csv("Expanded_Destinations.csv")
df2 = pd.read_csv("Final_Updated_Expanded_Reviews.csv")
df3 = pd.read_csv("Final_Updated_Expanded_UserHistory.csv")
df4 = pd.read_csv("Final_Updated_Expanded_Users.csv")

# Concatenate datasets
data = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Fill missing values
for col in data.select_dtypes(include=['number']).columns:
    data[col].fillna(data[col].mean(), inplace=True)
for col in data.select_dtypes(include=['object']).columns:
    data[col].fillna(data[col].mode()[0], inplace=True)

# Encode categorical columns
label_encoders = {}
categorical_cols = ['State', 'Type', 'BestTimeToVisit', 'Name']
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Standardize numeric columns
numeric_cols = ['Popularity', 'Rating']
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

# Features and Target
X = data[['State', 'Type', 'BestTimeToVisit', 'Popularity', 'Rating']]
y = data['Name']

# Train-test split
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in split.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

# Ensemble model
rf = RandomForestClassifier(n_estimators=200, max_depth=7, random_state=42)
gb = GradientBoostingClassifier(n_estimators=200, learning_rate=0.05, max_depth=4, random_state=42)
ensemble_model = VotingClassifier(estimators=[('rf', rf), ('gb', gb)], voting='soft')

# Fit and evaluate
ensemble_model.fit(X_train, y_train)
y_pred = ensemble_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# Save model
with open("model.pkl", "wb") as f:
    pickle.dump(ensemble_model, f)


/tmp/ipython-input-51-4214713004.py:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].mean(), inplace=True)
/tmp/ipython-input-51-4214713004.py:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)'

Accuracy: 0.75


In [52]:
# Enhanced Manual Recommendations (Complete and Balanced)
manual_recommendations = {
    "Goa": {
        "Beach": ["Calangute Beach", "Baga Beach"],
        "Historical": ["Basilica of Bom Jesus", "Chapora Fort"],
        "History": ["Fort Aguada", "Reis Magos Fort"],
        "Nature": ["Salim Ali Bird Sanctuary", "Dudhsagar Falls"],
        "Adventure": ["Scuba Diving at Grande Island", "Parasailing at Baga Beach"]
    },
    "Rajasthan": {
        "Beach": ["Mansagar Lake", "Ana Sagar Lake"],
        "Historical": ["Amber Fort", "City Palace"],
        "History": ["Hawa Mahal", "Jantar Mantar"],
        "Nature": ["Mount Abu", "Keoladeo National Park"],
        "Adventure": ["Desert Safari in Jaisalmer", "Hot Air Ballooning in Jaipur"]
    },
    "Kerala": {
        "Beach": ["Varkala Beach", "Kovalam Beach"],
        "Historical": ["Mattancherry Palace", "Bekal Fort"],
        "History": ["Fort Kochi", "Hill Palace"],
        "Nature": ["Munnar", "Alleppey Backwaters"],
        "Adventure": ["Trekking in Wayanad", "Boating in Periyar Lake"]
    },
    "Uttar Pradesh": {
        "Beach": ["Sangam Ghat (Prayagraj)", "Varanasi Ghats"],
        "Historical": ["Taj Mahal", "Fatehpur Sikri"],
        "History": ["Chunar Fort", "Rumi Darwaza"],
        "Nature": ["Dudhwa National Park", "Vindhyachal"],
        "Adventure": ["Wildlife Safari in Katarniaghat", "Rock Climbing in Sonbhadra"]
    },
    "Jammu and Kashmir": {
        "Beach": ["Manasbal Lake", "Wular Lake"],
        "Historical": ["Hari Parbat Fort", "Amar Mahal Palace"],
        "History": ["Mughal Gardens", "Shankaracharya Temple"],
        "Nature": ["Gulmarg", "Sonamarg"],
        "Adventure": ["Trekking in Pahalgam", "Skiing in Gulmarg"]
    }
}


In [56]:
# Gradio interface function
app_code = '''
def recommend_places(state, place_type, best_time):
     # Check if all selections are None
    if state == "None" and place_type == "None" and best_time == "None":
        return "Please select at least one option to get recommendations."

    # Convert input text to encoded values
    if state == "None":
        state_encoded = None
    elif state in label_encoders['State'].classes_:
        state_encoded = label_encoders['State'].transform([state])[0]
    else:
        return "Invalid state selection."

    if place_type == "None":
        type_encoded = None
    elif place_type in label_encoders['Type'].classes_:
        type_encoded = label_encoders['Type'].transform([place_type])[0]
    else:
        return "Invalid place type selection."

    if best_time == "None":
        best_time_encoded = None
    elif best_time in label_encoders['BestTimeToVisit'].classes_:
        best_time_encoded = label_encoders['BestTimeToVisit'].transform([best_time])[0]
    else:
        return "Invalid best time selection."

    # Filter based on given inputs
    filtered_data = data.copy()
    if state_encoded is not None:
        filtered_data = filtered_data[filtered_data['State'] == state_encoded]
    if type_encoded is not None:
        filtered_data = filtered_data[filtered_data['Type'] == type_encoded]
    if best_time_encoded is not None:
        filtered_data = filtered_data[filtered_data['BestTimeToVisit'] == best_time_encoded]

    # If no recommendations found, use manual fallback
    if filtered_data.empty:
        if state in manual_recommendations and place_type in manual_recommendations[state]:
            return manual_recommendations[state][place_type]
        else:
            return "No recommendations found. Try different selections."

    # Decode the recommended place names
    recommendations = label_encoders['Name'].inverse_transform(filtered_data['Name'].unique())

    return recommendations if len(recommendations) > 0 else "No recommendations found."
import gradio as gr
demo = gr.Interface(
    fn=recommend_places,
    inputs=[
        gr.Dropdown(choices=["None"] + list(label_encoders['State'].inverse_transform(range(len(label_encoders['State'].classes_)))), label='State'),
        gr.Dropdown(choices=["None"] + list(label_encoders['Type'].inverse_transform(range(len(label_encoders['Type'].classes_)))), label='Type'),
        gr.Dropdown(choices=["None"] + list(label_encoders['BestTimeToVisit'].inverse_transform(range(len(label_encoders['BestTimeToVisit'].classes_)))), label='Best Time to Visit'),
    ],
    outputs="text",
    title="Travel Destination Recommender",
    description="Get recommended travel destinations based on your selected state, place type, and best time to visit."
)

# Launch the app
if __name__ == "__main__":
    demo.launch()
'''

with open("app.py", "w") as f:
    f.write(app_code)
